# **Классификация с помощью сверточной нейронной сети**

## **Подготовка**

### **Сделаем, чтобы тесты были воспроиводимы на одном notebook(одинаковые нач состояния)**

In [ ]:
import torch
import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.determenistic = True

### **Импорт датасет**

In [ ]:
import torchvision.datasets
MNIST_train = torchvision.datasets.MNIST('./', download=True, train=True)
MNIST_test = torchvision.datasets.MNIST('./', download=True, train=False)

In [ ]:
X_train = MNIST_train.train_data
y_train = MNIST_train.train_labels
X_test = MNIST_test.test_data
y_test = MNIST_test.test_labels;

### **Узнать тип**

In [ ]:
X_train.dtype, y_train.dtype

### **Конвертация в флоат**

In [ ]:
X_train = X_train.float()
X_test = X_test.float()

### **Узнать форму**

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

### **Отрисовка одной цифры**

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_train[0, :, :])
plt.show()
print(y_train[0])

### **Изменим размерность**

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.unsqueeze(1).float()
X_test = X_test.unsqueeze(1).float()

# **Нейросеть и обучение**

### **Нейросеть**

In [ ]:
class LeNet5(torch.nn.Module):
  def __init__(self):
    super(LeNet5, self).__init__()
    self.conv1 = torch.nn.Conv2d(
       in_channels = 1, out_channels=6, kernel_size=5, padding=2)
    self.act1 = torch.nn.ReLU()
    self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv2 = torch.nn.Conv2d(
       in_channels = 6, out_channels=16, kernel_size=5, padding=0)
    self.act2 = torch.nn.ReLU()
    self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc1 = torch.nn.Linear(5*5*16, 120)
    self.act3 = torch.nn.Sigmoid()

    self.fc2 = torch.nn.Linear(120, 84)
    self.act4 = torch.nn.Sigmoid()

    self.fc3 = torch.nn.Linear(84, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = self.act1(x)
    x = self.pool1(x)

    x = self.conv2(x)
    x = self.act2(x)
    x = self.pool2(x)

    x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

    x = self.fc1(x)
    x = self.act3(x)
    x = self.fc2(x)
    x = self.act4(x)
    x = self.fc3(x)

    return x


lenet5_net = LeNet5()

In [ ]:
torch.cuda.is_available() #видит ли торч гпу

In [ ]:
!nvidia-smi #узнать, есть ли вычисления на видеокарте

In [ ]:
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

lenet5_net = lenet5_net.to(device)

### **Функция потерь и оптимизатор**

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(lenet5_net.parameters(), momentum=0.7, lr=0.01)

### **Обучение**

In [ ]:
batch_size = 100

test_accuracy_history = []
test_loss_history = []
# train_loss_history = []

X_test = X_test.to(device) # перекладываем на другой девайс
y_test = y_test.to(device) # перекладываем на другой девайс

for epoch in range(10000):
  order = np.random.permutation(len(X_train))
  for start_index in range(0, len(X_train), batch_size):
    optimizer.zero_grad()

    batch_indexes = order[start_index:start_index+batch_size]

    X_batch = X_train[batch_indexes].to(device) # приставка .to(device) см. выше
    y_batch = y_train[batch_indexes].to(device) # приставка .to(device) см. выше

    preds = lenet5_net.forward(X_batch)
    
    loss_value = loss(preds, y_batch)
    loss_value.backward()

    optimizer.step()

  test_preds = lenet5_net.forward(X_test)
  test_loss_history.append(loss(test_preds, y_test).data.cpu())

  accuracy = (test_preds.argmax(dim=1)==y_test).float().mean().data.cpu()
  test_accuracy_history.append(accuracy)

  print("Эпоха:{}, точность:{}".format(epoch,accuracy))

### **Графики точности и функции потерь**

In [ ]:
import matplotlib.pyplot as plt

import pylab



fig = plt.figure(figsize=(20, 6))
plt.subplot(121)
plt.plot(test_accuracy_history, c='r')
plt.title("accuracy")

plt.subplot(122)
plt.plot(test_loss_history)
#plt.plot(train_loss_history, c='g')
plt.title("loss")

plt.show()

### **Полезные команды**

In [ ]:
import pickle
with open('model.pickle', 'wb') as f:
  pickle.dump(lenet5_net, f)
# with open('model.pickle', 'rb') as f:
#   data_new = pickle.load(f)
from google.colab import files
files.download('model.pickle')
  